In [1]:
%matplotlib qt

import neuroseries as nts

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import scipy.ndimage
import os

import scipy.signal
import scipy.stats

import bk.load as bk
import bk.plot
import bk.compute

def TTLtoTimes(TTL,Fs = 20000):
    
    if isinstance(TTL[0],(np.bool_,bool)):
        TTL = list(map(int,TTL))
    
    diff_TTL = np.diff(TTL)
    
    t_start = np.where(diff_TTL == 1)[0]
    t_end = np.where(diff_TTL == -1)[0]
    t_TTL = np.array([np.mean(interval) for interval in zip(t_start,t_end)])
    
    return t_TTL/Fs

def speed(pos):
    
    x_speed = np.diff(pos['x'])/np.diff(pos.index)
    y_speed = np.diff(pos['y'])/np.diff(pos.index)
    
    v = np.sqrt(x_speed**2 + y_speed**2)
    
    return v

def load_pos():
    import pandas as pd
    files = os.listdir()
    for f in files: 
        if f.endswith('.h5'): 
            filename = f
    
    data = pd.read_hdf(filename)
    data = data[data.keys()[0][0]]
    return data

def load_digitalin(channels,Fs = 20000):
    import pandas as pd
    
    digital_word = np.fromfile('digitalin.dat','uint16')
    sample = len(digital_word)
    time = np.arange(0,sample)
    time = time/Fs

    chan = {}
    
    for i in range(len(channels)):
        chan.update({channels[i]:(digital_word & 2**i)>0})

    return time,chan

# def load_digitalin(path,nchannels=16,Fs = 20000):
#     import pandas as pd
    
#     digital_word = np.fromfile(path,'uint16')
#     sample = len(digital_word)
#     time = np.arange(0,sample)
#     time = time/Fs

    
#     for i in range(nchannels):
#         if i == 0: data = (digital_word & 2**i)>0
#         else: data = np.vstack((data,(digital_word & 2**i)>0))

#     return data



In [52]:
path = 'Y:/ExampleSession/rat01_200630_114732'
path = 'Y:/ExampleSession/rat01_200630_171900'
path = 'Y:/ExampleSession/rat01_200630_183149/'

path = 'Y:/ExampleSession/RAT02/rat02_200707_114611'


i = 1
title = ['Fear','Tone','Context']

os.chdir(path)
step = 1

time,data = load_digitalin(['TTL','Shock','Tone'])
pos = load_pos()

s = speed(pos['snout'])
s = scipy.ndimage.gaussian_filter1d(s,1)

tf = TTLtoTimes(data['TTL'])
time = time-tf[0]
tf = np.delete(tf,-1)

pos.index = tf-tf[0]
time = time[0:-1:step]

fig = plt.subplot(1,1,i)
plt.tight_layout()
plt.title(title[i-1])
plt.plot(time,data['TTL'][0:-1:step])
plt.plot(time,data['Shock'][0:-1:step] * 3)
plt.plot(time,data['Tone'][0:-1:step]*2)
plt.plot(pos.index[:-1],s/15)
plt.legend(['TTL','Shock','Tone','Speed'])
plt.xlabel('Time (s)')
#plt.vlines(x=tf,ymin = 0, ymax = 3,color='red')
i += 1

plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [54]:
pos.as_units('s')

coords,x,y,likelihood
Time (s),,,
0.000000,380.488586,186.74437,0.000012
0.033324,NaN,NaN,NaN
0.066675,NaN,NaN,NaN
0.099999,NaN,NaN,NaN
0.133324,NaN,NaN,NaN
...,...,...,...
617.954025,NaN,NaN,NaN
617.987350,NaN,NaN,NaN
618.020675,NaN,NaN,NaN


In [53]:
pos = nts.TsdFrame(t = pos.index.values,d = pos['snout'],time_units='s')


In [5]:
path = '/media/billel/DATA/ExampleSession/RAT01/'


In [13]:
path = '/media/billel/DATA/ExampleSession/RAT01/'
subpath = os.listdir(path)

In [140]:
i = 1
title = ['Fear','Tone','Context']


# os.chdir(os.path.join(path,subpath[1]))
os.chdir('Y:/ExampleSession/RAT01/rat01_200630_171900')
step = 20





time,data = load_digitalin(['TTL','Shock','Tone'])

tf = TTLtoTimes(data['TTL'])
tf = np.delete(tf,-1)

time = time-tf[0]

pos = load_pos()
pos.index = tf-tf[0]


s = speed(pos['snout'])
s = scipy.ndimage.gaussian_filter1d(s,1)
s = nts.Tsd(pos.index.values[:-1],s,time_units='s')


time = time[0:-1:step]
fig = plt.subplot(2,1,1)
plt.tight_layout()
plt.title(title[i-1])
plt.plot(time,data['TTL'][0:-1:step])
plt.plot(time,data['Shock'][0:-1:step] * 3)
plt.plot(time,data['Tone'][0:-1:step]*2)
plt.plot(pos.index[:-1],s.values/1500)
plt.legend(['TTL','Shock','Tone','Speed'])
plt.xlabel('Time (s)')
#plt.vlines(x=tf,ymin = 0, ymax = 3,color='red')
i += 1
freezing_intervals = bk.compute.freezing(s,50)
bk.plot.intervals(freezing_intervals,col='orange')

inter = nts.IntervalSet(starts,ends,time_units='s')
plt.subplot(2,1,2,sharex=fig)
bk.plot.intervals(inter,col='grey')

plt.show()

Z:\Python\Billel\neuroseries\interval_set.py:65: UserWarning: start and end not of the same length
  warn(msg, UserWarning)


In [139]:
tmin= 0.2
starts = []
ends = []
for i,z in enumerate(zip(freezing_intervals.as_units('s').start, freezing_intervals.as_units('s').end)):
    s=z[0]
    e=z[1]
    
    if ends:
        if s<ends[-1]: continue
    
    if freezing_intervals.as_units('s').start[i+1]-e<tmin:
        starts.append(s)
       # ends.append(freezing_intervals.as_units('s').end[i+1])
        j=i
        while freezing_intervals.as_units('s').start[j+1]-freezing_intervals.as_units('s').end[j]<tmin:
            j=j+1
        ends.append(freezing_intervals.as_units('s').end[j])
    else:
        starts.append(s)
        ends.append(e)


KeyError: 466

In [36]:
print(freezing_intervals)

         start        end
0       466550    2499375
1      1632925    4198950
2      2432725    5298675
3      3732400    5898525
4      5232025    6198450
5      5831875    6964925
6      6165125    7531450
7      6731650    7731400
8      7198200    8097975
9      7598100    8797800
10     7964675    9064400
11     8397900   10564025
12     8997750   11463800
13    10497375   12263600
14    11197200   12496875
15    11630425   12996750
16    12430225   14163125
17    12763475   14496375
18    14096475   15029575
19    14363075   17029075
20    14829625   17862200
21    16962425   18228775
22    17762225   18662000
23    18128800   19928350
24    18428725   21694575
25    19628425   22127800
26    21661250   22627675
27    22027825   22994250
28    22394400   24193950
29    22927600   24527200
..         ...        ...
436  387003225  393068375
437  390469025  393401625
438  390702300  394934575
439  393135025  396767450
440  393468275  398633650
441  394967900  401699550
442  3970340

In [30]:
def freezing(speed,treshold):
    """
        BK 8/11/20
        Input 
            speed: speed vector as output by bk.compute.speed (not yet implemented. But it's an nts.frame)
            treshold: arbritary units
    """
    
    fs =  1/scipy.stats.mode(np.diff(speed.index)).mode[0]
    freezing = speed.values<treshold
    if freezing[0] == 1: freezing[0] = 0
    if freezing[-1] == 1: freezing = np.append(freezing,0)
        
    dfreeze = np.diff(freezing.astype(np.int8))
    start = np.where(dfreeze == 1)[0]/fs + speed.index[0]
    end = np.where(dfreeze == -1)[0]/fs + speed.index[0]
    
    freezing_intervals = nts.IntervalSet(start,end)
    return freezing_intervals

In [6]:
speed = s 
treshold = 1

In [7]:
fs =  1/scipy.stats.mode(np.diff(speed.index)).mode[0]
freezing = speed.values<treshold
if freezing[0] == 1: freezing[0] = 0
if freezing[-1] == 1: freezing = np.append(freezing,0)

dfreeze = np.diff(freezing.astype(np.int8))

start = np.where(dfreeze == 1)[0]/fs + speed.index[0]
end = np.where(dfreeze == -1)[0]/fs + speed.index[0]

freezing_intervals = nts.IntervalSet(start,end,time_units = 's')

Empty IntervalSet
Columns: [start, end]
Index: []


In [9]:
speed.values

array([ 269.38943355, 1118.12733579, 2096.431932  , ...,   10.30124845,
         13.91425449,   15.7500021 ])

In [15]:
del freezing

In [57]:
path = 'Y:/ExampleSession/RAT01/rat01_200630_114732/'
os.chdir(path)

In [64]:
t_TTL = load_digitalin(['TTL'])

In [72]:
t_TTL = TTLtoTimes(t_TTL[1]['TTL'])

array([ 20.675375,  20.708725,  20.74205 , ..., 618.32965 , 618.362975,
       618.3827  ])

In [87]:
pos = load_pos()
t_TTL = np.delete(t_TTL,-1)
pos.index = t_TTL - t_TTL[0]

In [101]:
s = speed(pos['snout'])
s = nts.Tsd(t=pos.index.values[:-1],d = s,time_units='s')

In [95]:
plt.plot(s)

In [104]:
pos['snout']
s.as_units('s')

Time (s)
0.000000       7.832967
0.033350       8.782744
0.066675      22.644989
0.100000      14.177008
0.133350      14.646234
                ...    
597.520950     1.695441
597.554275    10.491249
597.587625    10.262282
597.620950    11.373562
597.654275     7.952450
Length: 17931, dtype: float64

In [111]:
f = bk.compute.freezing(s,20)

In [110]:
plt.plot(s.as_units('s'))

In [113]:
bk.plot.intervals(f,col= 'orange')

In [224]:
time,data = load_digitalin(['TTL','Shock','Tone'])

tf = TTLtoTimes(data['TTL'])
tf = np.delete(tf,-1)

time = time-tf[0]

pos = load_pos()
pos.index = tf-tf[0]
step = 20

s = speed(pos['snout'])
size = 20
gaussian = scipy.signal.gaussian(size,30)
s = np.convolve(s,gaussian,'same')/size

# s = scipy.ndimage.gaussian_filter1d(s,20)
s = nts.Tsd(pos.index.values[:-1],s,time_units='s')

plt.figure()
time = time[0:-1:step]
fig = plt.subplot(1,1,1)
plt.tight_layout()
plt.title(title[i-1])
plt.plot(time,data['TTL'][0:-1:step])
plt.plot(time,data['Shock'][0:-1:step] * 3)
plt.plot(time,data['Tone'][0:-1:step]*2)
plt.plot(pos.index[:-1],s.values/1500)
plt.legend(['TTL','Shock','Tone','Speed'])
plt.xlabel('Time (s)')
#plt.vlines(x=tf,ymin = 0, ymax = 3,color='red')


bk.plot.intervals(bk.compute.freezing(s,20),col='orange')
plt.show()

In [225]:
plt.figure()
plt.plot(scipy.signal.gaussian(500,5))


In [226]:
s.index

Int64Index([        0,     33350,     66675,    100000,    133350,    166675,
               199999,    233350,    266674,    300000,
            ...
            597354300, 597387625, 597420950, 597454274, 597487625, 597520950,
            597554275, 597587625, 597620950, 597654275],
           dtype='int64', name='Time (us)', length=17931)

In [129]:
s = speed(pos['snout'])

In [130]:
s.mean()

48.77639213044943

In [133]:
s.values

array([431.60497243, 443.29986216, 454.99992449, ..., 612.47055474,
       591.45811707, 570.76316348])

In [134]:
plt.plot(s.values)

In [148]:
y = np.sin(np.arange(0,6,0.01))

In [209]:
y = speed(pos['snout'])
gaussian = scipy.signal.gaussian(10,10)



In [221]:
y = np.sin(np.arange(0,6,0.01))
np_conv = np.convolve(y,gaussian,'same')/10
scipy_conv = scipy.signal.fftconvolve(y,gaussian,'same')
scipy_filter = scipy.ndimage.gaussian_filter1d(y,10)

In [222]:
plt.plot(y)
plt.plot(np_conv)
plt.plot(scipy_conv)
plt.plot(scipy_filter)

plt.legend(['ori','np','scipy conv','scipy ndimage'])

In [2]:
def consolidateIntervals(intervals, tmin):
    
    for i in intervals:
        print(i)
    start = []
    end = []
    
  #  intervals_consolidated = nts.Interval_Set(start,end,time_unit ='s')



In [ ]:
scipy.ndimage.gaussian_filter1d()